In [7]:
pwd

'/home/wsuser/work'

In [8]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten

In [10]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='oIbr0fBehUOMTXjyGQ7PuqVqDDIHvLGdxRmHf9bz_Rxr',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'communicationdeadanddumb-donotdelete-pr-wi3aitewfbnvl0'
object_key = 'conversation engine for deaf and dumb.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
if not hasattr(streaming_body_1, "__iter__"):streaming_body_1.__iter__ = types.MethodType(__iter__,streaming_body_1)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [11]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [12]:
import os
filenames = os.listdir('/home/wsuser/work/conversation engine for deaf and dumb/training_set')

In [13]:
X_train = train_datagen.flow_from_directory('/home/wsuser/work/conversation engine for deaf and dumb/training_set', target_size = (64,64), batch_size = 300, class_mode = 'categorical', color_mode = 'grayscale')
X_test = test_datagen.flow_from_directory('/home/wsuser/work/conversation engine for deaf and dumb/test_set', target_size = (64,64), batch_size = 300, class_mode = 'categorical', color_mode = 'grayscale')

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [14]:
X_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

In [15]:
model = Sequential()
model.add(Convolution2D(32,(3,3), input_shape = (64,64,1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 9, activation = 'softmax'))

In [16]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
model.fit(X_train, steps_per_epoch = len(X_train),epochs=10, validation_data = X_test, validation_steps= len(X_test))

Epoch 1/10
53/53 [==============================] - 146s 3s/step - loss: 0.7353 - accuracy: 0.7479 - val_loss: 0.3811 - val_accuracy: 0.9191
Epoch 2/10
53/53 [==============================] - 142s 3s/step - loss: 0.1022 - accuracy: 0.9708 - val_loss: 0.1659 - val_accuracy: 0.9653
Epoch 3/10
53/53 [==============================] - 144s 3s/step - loss: 0.0437 - accuracy: 0.9877 - val_loss: 0.1562 - val_accuracy: 0.9764
Epoch 4/10
53/53 [==============================] - 138s 3s/step - loss: 0.0235 - accuracy: 0.9937 - val_loss: 0.1719 - val_accuracy: 0.9769
Epoch 5/10
53/53 [==============================] - 136s 3s/step - loss: 0.0155 - accuracy: 0.9957 - val_loss: 0.1818 - val_accuracy: 0.9773
Epoch 6/10
53/53 [==============================] - 139s 3s/step - loss: 0.0116 - accuracy: 0.9969 - val_loss: 0.1925 - val_accuracy: 0.9778
Epoch 7/10
53/53 [==============================] - 139s 3s/step - loss: 0.0073 - accuracy: 0.9980 - val_loss: 0.1998 - val_accuracy: 0.9693
Epoch 8/10
53

In [18]:
model.save('conv.h5')

In [19]:
!tar -zcvf conversation_engine.tgz conv.h5

conv.h5


In [20]:
ls -1

'conversation engine for deaf and dumb'/
conversation_engine.tgz
conv.h5


In [21]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 12.7 MB/s eta 0:00:01


In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "e9o1OIOfeMwU3gAmqbrwN2bLpGYDJ7ZdLCgcaz0biAof"
}
client = APIClient(wml_credentials)

In [25]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [26]:
space_uid = guid_from_space_name(client, 'conv_engine')
print("Space UID = " + space_uid)

Space UID = 1233b3f7-5150-422d-a41b-523e665492e0


In [28]:
client.set.default_space(space_uid)

'SUCCESS'

In [41]:
!pip install keras==2.2.4

     |████████████████████████████████| 312 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 9.6 MB/s  eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.0
    Uninstalling Keras-2.4.0:
      Successfully uninstalled Keras-2.4.0


In [42]:
client.software_specifications.list(limit =100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [33]:
import keras

In [34]:
keras.__version__

'2.7.0'

In [30]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.8-horovod")
software_spec_uid

'217c16f6-178f-56bf-824a-b19f20564c49'

In [39]:
model_details = client.repository.store_model(model = 'conversation_engine.tgz',meta_props = {
client.repository.ModelMetaNames.NAME:"CV",
client.repository.ModelMetaNames.TYPE:"keras_2.7.0",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

Failure during creating model. (POST https://us-south.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=1233b3f7-5150-422d-a41b-523e665492e0)
Status code: 400, body: {
  "trace": "b4850d41f0790f405cfe2281dd9ee172",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported model type 'keras_2.7.0'",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


ApiRequestFailure: Failure during creating model. (POST https://us-south.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=1233b3f7-5150-422d-a41b-523e665492e0)
Status code: 400, body: {
  "trace": "b4850d41f0790f405cfe2281dd9ee172",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported model type 'keras_2.7.0'",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}